# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [2]:
import pandas as pd
import os
import numpy as np

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [3]:
!cat talks.tsv







































1	Neutrino trident production at near detectors	invited seminar		Fermilab Theory Seminar	05/18	USA	https://theory.fnal.gov/events/event/title-tba-neutrino-3/	

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [4]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,ID,title,type,url_slug,venue,date,location,talk_url,description
0,38,NaN,invited seminar,NaN,McGill University,05/21,Canada,NaN,Virtual
1,37,NaN,invited seminar,NaN,"ETH, Zurich",04/21,Switzerland,NaN,Virtual
2,36,NaN,invited seminar,NaN,"C3P, UCLouvain",04/21,Belgium,NaN,Virtual
3,35,NaN,parallel talk,NaN,American Physics Society April Meeting,04/21,USA,NaN,Virtual
4,34,NaN,invited seminar,NaN,Carleton University,04/21,Canada,NaN,Virtual
5,33,NaN,invited seminar,NaN,University of California Santa Cruz,03/21,USA,NaN,Virtual
6,32,NaN,invited seminar,NaN,"Neutrino Seminar, Fermilab",03/21,USA,NaN,Virtual
7,31,NaN,parallel talk,NaN,XIX International Workshop on Neutrino Telescopes,02/21,Italy,NaN,Virtual
8,30,Snowmass Dark Sector Studies at High Intensiti...,invited talk,NaN,Snowmass Dark Sector Studies at High Intensiti...,12/20,USA,NaN,Virtual
9,29,NaN,plenary talk,NaN,"3rd South American Dark Matter Workshop, ICTP",10/20,Brazil,NaN,Virtual


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [5]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

In [6]:
talks = talks.replace(np.nan, '', regex=True)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [11]:
loc_dict = {}

for row, item in talks.iterrows():
    year = f'20{item.date[2:]}'
    month = item.date[:2]
    if len(str(item.ID)) == 1:
        id_for_sorting = f'000{item.ID}'
    elif len(str(item.ID)) == 2:
        id_for_sorting = f'00{item.ID}'
    elif len(str(item.ID))==3:
        id_for_sorting = f'0{item.ID}'
    else:
        print(f"Invalid ID number {item.ID} with length {len(str(item.ID))}")
        break;
    md_filename = f"{str(id_for_sorting)}_{item.venue}.md"
    html_filename = str(id_for_sorting)
    md = "---\n"
    md += "collection: talks" + "\n"
    md += 'talk_number: "' + str(item.ID) + '"\n'
    md += 'id_for_sorting: "' + str(id_for_sorting) + '"\n'
    md += "permalink: /talks/" + html_filename + "\n"
       
    
    if len(str(item.type)) > 1:
        md += f"title: {item.title} \n"
    
          
    if len(str(item.type)) > 0:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "talk"\n'
    
    if len(str(item.venue)) > 0:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.date)) > 0:
        md += "date: " + str(item.date) + "\n"
    if len(str(item.location)) > 0:
        md += 'location: "' + str(item.location) + '"\n'
        
    if len(str(item.talk_url)) > 3:
        md += f"link: True \n"         
        md += f"talk_url: {str(item.talk_url)} \n" 
                   
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [51]:
!ls ../_talks

18_05_Fermilab Theory Seminar.md
18_05_Phenomenology Symposium 2018.md
18_06_Near detector workshop 2018, CERN.md
18_06_Perimeter Institute.md
18_08_NuFact 2018, Virginia.md
18_09_Neutrino Oscillation Workshop 2018.md
18_11_Max-Planck-Institut fur Kernphysik, Heidelberg.md
18_12_Physics Opportunities at the Near Detector of DUNE (PONDD), Fermilab.md
19_03_Queen Mary University of London.md
19_04_Prospects of Neutrino Physics, IPMU.md
19_05_IFIC, Valencia.md
19_05_Neutrino Theory Network Workshop, Washington U., St Louis.md
19_06_Invisibles Workshop 2019, Valencia.md
19_08_Columbia University.md
19_08_MicroBooNE collaboration call.md
19_12_NuPhys 2019.md
20_02_Fermilab Theory Seminar.md
20_05_Brookhaven Neutrino Theory Virtual Seminars, BNL.md
20_05_Phenomenology Symposium 2020, Pittsburgh.md
20_06_JGU Theorie Palaver, Mainz.md
20_06_Neutrino 2020, University of Chicago.md
20_07_ICHEP 2020.md
20_09_Snowmass Neutrino Frontier 03 kick-off meeting.md
20_09_Snowmass Theory of neutrino physi

In [25]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
